# Building a Simple Agent with LangGraph

This notebook demonstrates how to build a conversational agent with search capabilities using LangGraph. We'll create an agent that can perform web searches to answer questions, making it more capable than a standalone LLM.

## Setup and Installation

First, we install the required packages:

In [85]:
!pip install langchain langgraph openai pygraphviz

  Using cached pygraphviz-1.14.tar.gz (106 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pygraphviz (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [100 lines of output]
      /private/var/folders/v0/pct0272j5r1fspj35flrgxzw0000gn/T/pip-build-env-s3lu98yb/overlay/lib/python3.13/site-packages/setuptools/config/_apply_pyprojecttoml.py:82: SetuptoolsDeprecationWarning: `project.license` as a TOML table is deprecated
      !!
      
              ********************************************************************************
              Please use a simple string containing a SPDX expression for `project.license`. You can also use `project.license-files`.
      
              By 2026-Feb-18, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
      


**Note**: The installation of `pygraphviz` might fail depending on your system configuration. This doesn't prevent our agent from working - it just means we won't be able to visualize the graph structure.

## Environment Setup

We load environment variables from a `.env` file, which contains API keys for OpenAI and Tavily:

In [ ]:
from dotenv import load_dotenv
_ = load_dotenv()
print(_)

## Importing Dependencies

We import the necessary libraries for building our agent:

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
import json
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

Key imports explained:
- `StateGraph` and `END` from langgraph: Used for creating the agent's workflow
- Message types from LangChain: Provide structured communication formats
- `ChatOpenAI`: For accessing OpenAI language models
- `TavilySearchResults`: Provides web search capability

## Setting Up the Search Tool

We initialize the Tavily search tool that our agent will use to look up information online:

In [ ]:
tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)

We configure it to return 4 results per search to give our agent more context. The `tool.name` will show us the name that the LLM needs to use when calling this tool.

## Defining the Agent State

We create a typed dictionary class that will track the agent's state - specifically the messages in the conversation:

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

The `Annotated` with `operator.add` is important here - it specifies that when the state is updated, new messages should be appended to the existing list rather than replacing it. This allows our conversation to build up over time.

## Agent Implementation

Now we implement the `Agent` class that orchestrates the conversation flow:

In [ ]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

Let's break down how this agent works:

### Graph Structure

The agent is structured as a simple directed graph with two nodes:
- `llm`: Calls the language model to generate responses or tool calls
- `action`: Executes any tool calls the LLM requests

### Flow Logic
1. The LLM is called first with the user's input
2. If the LLM wants to use a tool (determined by `exists_action`), we go to the action node
3. After executing the tool, we go back to the LLM to interpret the results
4. If the LLM doesn't need a tool, we end the flow and return its direct response

### Key Methods
- `exists_action`: Determines if the LLM wants to use a tool
- `call_openai`: Calls the LLM with the conversation history
- `take_action`: Executes any tool calls and returns the results

### Error Handling
There's defensive code to handle cases where the LLM tries to call a tool that doesn't exist.

## Agent Configuration

We set up our agent with a system prompt that guides its behavior:

In [ ]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="gpt-3.5-turbo")  #reduce inference cost
abot = Agent(model, [tool], system=prompt)

The system prompt instructs the agent to use the search tool intelligently and only when necessary. We initially use the more cost-effective GPT-3.5 Turbo model.

## Attempted Graph Visualization

We can try to visualize the agent's graph structure, but this will only work if `pygraphviz` was successfully installed:

In [ ]:
from IPython.display import Image

try:
    Image(abot.graph.get_graph().draw_png())
except ImportError:
    print("Could not visualize graph - pygraphviz not installed correctly")

## Testing the Agent with Simple Queries

Let's test the agent with a weather-related query, which will trigger a web search:

In [ ]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

In [ ]:
# View the entire result object
result

In [ ]:
# View just the final response content
result['messages'][-1].content

## Testing with Multiple Cities

Let's see how the agent handles multiple locations in a single query:

In [ ]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

In [ ]:
result['messages'][-1].content

## Testing with Complex Multi-Step Questions

Now let's try a more complex query that requires multiple searches and reasoning steps. 
For this, we'll switch to the more powerful GPT-4o model:

In [ ]:
# Note, the query was modified to produce more consistent results. 
# Results may vary per run and over time as search information and models change.

query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question." 
messages = [HumanMessage(content=query)]

model = ChatOpenAI(model="gpt-4o")  # requires more advanced model
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

In [ ]:
print(result['messages'][-1].content)

## Analysis of the Multi-Step Query

In this complex query, the agent had to:

1. Find out who won the Super Bowl in 2024 (Kansas City Chiefs)
2. Determine which state their headquarters is in (Missouri)
3. Look up the GDP of that state (~$454.6 billion)

The agent successfully chained these steps together without human intervention, demonstrating the power of this approach.

## Conclusion and Key Insights

This notebook has demonstrated several key concepts in building AI agents with LangGraph:

1. **Directed Graphs for Workflows**: LangGraph makes it easy to create clear workflows for agent behavior
2. **Tool Integration**: The framework provides a clean way to integrate tools like web search
3. **Conditional Logic**: We can implement decision-making using conditional edges
4. **State Management**: The graph handles tracking the conversation state automatically
5. **Defensive Programming**: The implementation includes error handling for LLM mistakes
6. **Model Selection**: More complex reasoning tasks benefit from more advanced models

By combining these elements, we've created a conversational agent that can access external information and manage multi-step reasoning processes.

## Further Improvements

The agent could be enhanced in several ways:

1. Adding more tools beyond just search
2. Implementing memory to remember previous conversations
3. Adding planning capabilities for more complex tasks
4. Improving error handling and recovery
5. Adding user feedback mechanisms

LangGraph provides a flexible framework for implementing these enhancements.